In [5]:
# Libraries
import pandas as pd

In [106]:
# CSV Sources

properties_cols = ['property_id','latitude','longitude','localidad', 'city', 'country']
df_properties = pd.read_csv('sources\properties.csv', usecols = properties_cols, converters = {'localidad': lambda x: (x.replace('\xad','').title())}) # converters used to clean the locality column, Chí­A -> Chí­a

users_cols = ['property_id','user_id']
df_users = pd.read_csv('sources\\users.csv', usecols = users_cols)

users_cols = ['scheduled_id','property_id','begin_date','end_date','status']
df_visits = pd.read_csv('sources\\visits.csv', usecols = users_cols)